In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn import (linear_model, model_selection)
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [18]:
games = pd.read_csv("prepared/combined/games-all.csv")
playerStats = pd.read_csv("prepared/combined/player-stats-all.csv")
table = pd.read_csv("prepared/combined/table-all.csv")
teamStats = pd.read_csv("prepared/combined/team-stats-all.csv")
df = pd.merge(games, table)
df = pd.merge(df, playerStats)
#df = pd.merge(df, teamStats)

In [19]:
y = df["Rk"]

X = df
X.drop(["Rk"], axis=1)
X = pd.get_dummies(X)


In [20]:
X_train, X_valtest, y_train, y_valtest = model_selection.train_test_split(X, y, test_size=0.4,random_state=42)
X_valid, X_test, y_valid, y_test = model_selection.train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=42)

In [26]:
models = {'kNN5': KNeighborsRegressor(n_neighbors=5),
          'kNN7': KNeighborsRegressor(n_neighbors=7),
          'kNN9': KNeighborsRegressor(n_neighbors=9),
          'kNN11': KNeighborsRegressor(n_neighbors=11),
          'LinR': linear_model.LinearRegression(),
          'RFor': RandomForestRegressor(random_state=0)}

bestmodel = None
bestmodelscore = 10
print("RMSE Score:")
for key, model in models.items():
    model = model.fit(X_train, y_train)
    score = mean_squared_error(y_valid, model.predict(X_valid), squared=True)
    if score<bestmodelscore:
        bestmodelscore = score
        bestmodel = model
    print(f"{key}: {score:.5f}")

RMSE Score:
kNN5: 9.14400
kNN7: 9.57551
kNN9: 9.60988
kNN11: 9.48926
LinR: 0.00278
RFor: 0.75068


In [25]:
scores = {key: r2_score(y_valid, model.predict(X_valid)) for key, model in models.items()}
scoredf = pd.DataFrame(scores, index=['r2 score']).transpose().sort_values(by='r2 score', ascending=False)
print(scoredf)

       r2 score
LinR   0.999679
RFor   0.913116
kNN1  -0.058333
kNN50 -0.098294
kNN10 -0.108277
kNN20 -0.112254


In [49]:
games20 = pd.read_html("2020/games.xls", encoding="UTF-8")
games20 = pd.DataFrame(games20[0]).dropna(how="all")

games20 = pd.get_dummies(games20)

,Wk,Date,Time,Home,Away,Venue
1,1.0,2020-07-03,19:45,Trondheims-Ørn,LSK Kvinner,Ørn Arena
2,1.0,2020-07-04,11:30,Røa,Arna-Bjørnar,Røabanen
3,1.0,2020-07-04,13:00,Avaldsnes,Lyn,Avaldsnes Idrettssenter
4,1.0,2020-07-04,14:00,Sandviken,Vålerenga,Stemmemyren kunstgressbane
5,1.0,2020-07-05,15:00,Klepp,Kolbotn,Klepp Stadion
...,...,...,...,...,...,...
107,NaN,2020-11-25,19:00,Medkila,Kolbotn,Harstad Stadion
108,18.0,2020-12-06,13:00,Klepp,Trondheims-Ørn,Klepp Stadion
109,18.0,2020-12-06,13:00,Sandviken,Avaldsnes,Stemmemyren kunstgressbane
110,18.0,2020-12-06,13:00,LSK Kvinner,Lyn,LSK-hallen


In [50]:
print(bestmodel.predict(games20))

ValueError: could not convert string to float: '2020-07-03'